In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 데이터 불러오기

In [23]:
from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
!cp "/content/drive/MyDrive/BOAZ/dirty_mnist_2nd.zip" "dirty_mnist_2nd.zip"
# data_2.zip을 현재 디렉터리에 압축해제
!unzip "dirty_mnist_2nd.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: 45000.png               
 extracting: 45001.png               
 extracting: 45002.png               
 extracting: 45003.png               
 extracting: 45004.png               
 extracting: 45005.png               
 extracting: 45006.png               
 extracting: 45007.png               
 extracting: 45008.png               
 extracting: 45009.png               
 extracting: 45010.png               
 extracting: 45011.png               
 extracting: 45012.png               
 extracting: 45013.png               
 extracting: 45014.png               
 extracting: 45015.png               
 extracting: 45016.png               
 extracting: 45017.png               
 extracting: 45018.png               
 extracting: 45019.png               
 extracting: 45020.png               
 extracting: 45021.png               
 extracting: 45022.png               
 extracting: 45023.png               
 extracting: 45024.png               
 extracting: 4

In [20]:
from google.colab import output
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
# 출력 결과 지우기
output.clear()

# 2. 라이브러리 임포트

In [2]:
!pip install torchinfo

In [3]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

from torchvision import transforms
from torchvision.models import resnet50

## 1. 커스텀 데이터셋 만들기

In [4]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir,
        image_ids,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

## 2. 이미지 어그멘테이션

In [5]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]),
    transforms.RandomRotation(60, expand=False),
    transforms.RandomAffine(30)
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]),
    transforms.RandomRotation(60, expand=False),
    transforms.RandomAffine(30)
])

In [6]:
#trainset = MnistDataset("./dirty_mnist/", 'dirty_mnist_2nd_answer.csv', transforms_train)
#testset = MnistDataset('./test_dirty_mnist/', 'sample_submission.csv', transforms_test)

#train_loader = DataLoader(trainset, batch_size=64, num_workers=8)
#test_loader = DataLoader(testset, batch_size=32, num_workers=4)

In [9]:
trainset = MnistDataset('/content/drive/MyDrive/BOAZ/dirty_mnist_2nd/', '/content/drive/MyDrive/BOAZ/dirty_mnist_2nd_answer.csv', transforms_train)
testset = MnistDataset('/content/drive/MyDrive/BOAZ/test_dirty_mnist/', '/content/drive/MyDrive/BOAZ/sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset, batch_size=64, num_workers=8)
#train_loader = DataLoader(trainset, batch_size=256, num_workers=8)
#test_loader = DataLoader(testset, batch_size=32, num_workers=4)
test_loader = DataLoader(testset, batch_size=32, num_workers=4)

## 3. ResNet50 모형

In [10]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



Layer (type:depth-idx)                   Output Shape              Param #
├─ResNet: 1-1                            [1, 1000]                 --
|    └─Conv2d: 2-1                       [1, 64, 128, 128]         9,408
|    └─BatchNorm2d: 2-2                  [1, 64, 128, 128]         128
|    └─ReLU: 2-3                         [1, 64, 128, 128]         --
|    └─MaxPool2d: 2-4                    [1, 64, 64, 64]           --
|    └─Sequential: 2-5                   [1, 256, 64, 64]          --
|    |    └─Bottleneck: 3-1              [1, 256, 64, 64]          75,008
|    |    └─Bottleneck: 3-2              [1, 256, 64, 64]          70,400
|    |    └─Bottleneck: 3-3              [1, 256, 64, 64]          70,400
|    └─Sequential: 2-6                   [1, 512, 32, 32]          --
|    |    └─Bottleneck: 3-4              [1, 512, 32, 32]          379,392
|    |    └─Bottleneck: 3-5              [1, 512, 32, 32]          280,064
|    |    └─Bottleneck: 3-6              [1, 512, 32, 32] 

## 4. 학습하기

In [12]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 30 # 10 epoch
model.train()

for epoch in range(num_epochs):
    # cuda cache 초기화
    torch.cuda.empty_cache()
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.type(torch.FloatTensor).to(device)
        targets = targets.type(torch.FloatTensor).to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()
    
        if (i+1) % 50 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value i

0: 0.69430, 0.54627
0: 0.69046, 0.54447
0: 0.69043, 0.53846
0: 0.69270, 0.53365
0: 0.69127, 0.54026
0: 0.69269, 0.53065
0: 0.69159, 0.54387
0: 0.69332, 0.52885
0: 0.68882, 0.53726
0: 0.69179, 0.53606
0: 0.68844, 0.54026
0: 0.68827, 0.53606
0: 0.68815, 0.53666
0: 0.68754, 0.53846
0: 0.68874, 0.53365


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value i

1: 0.68918, 0.54808
1: 0.68554, 0.54507
1: 0.68793, 0.53846
1: 0.68891, 0.53726
1: 0.68727, 0.53966
1: 0.68868, 0.53125
1: 0.68605, 0.54447
1: 0.68862, 0.53065
1: 0.68866, 0.53786
1: 0.68824, 0.53726
1: 0.68612, 0.54267
1: 0.68857, 0.53606
1: 0.68695, 0.53666
1: 0.68752, 0.53846
1: 0.68795, 0.53245


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value i

...

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value i

22: 0.34554, 0.84495
22: 0.32862, 0.84916
22: 0.41351, 0.80950
22: 0.36598, 0.83594
22: 0.36278, 0.83353
22: 0.34435, 0.84856
22: 0.37845, 0.82873
22: 0.38016, 0.83594
22: 0.37322, 0.83053
22: 0.37085, 0.82813
22: 0.36250, 0.83654
22: 0.36700, 0.83173
22: 0.39184, 0.82512
22: 0.37050, 0.83654
22: 0.38642, 0.82332


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value i

23: 0.32649, 0.85757
23: 0.34004, 0.84796
23: 0.38967, 0.82572
23: 0.36078, 0.84435
23: 0.37774, 0.83534
23: 0.34227, 0.85036
23: 0.34942, 0.84675
23: 0.34906, 0.84555
23: 0.38742, 0.81791
23: 0.36697, 0.82993
23: 0.34162, 0.84435
23: 0.36931, 0.84075
23: 0.38683, 0.82332
23: 0.38107, 0.82572
23: 0.36342, 0.83053


KeyboardInterrupt: ignored

## 5. 추론하기

In [28]:
submit = pd.read_csv('/content/drive/MyDrive/BOAZ/test_dirty_mnist/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('/content/drive/MyDrive/Resnet50_epoch30_batch64_rotation_affine_submit.csv', index=False)

FileNotFoundError: ignored